# Mapping with plotly express
plotly.express

In [ ]:
import boto3
from boto3.dynamodb.conditions import Key, Attr
import pandas as pd
import plotly.offline as pyo
import plotly.express as px

from datetime import datetime, timedelta
offset_hrs=-11

def get_utc_string(from_tz):
    from_tz_utc = datetime.strptime(from_tz, "%Y-%m-%d %H:%M") + timedelta(hours=offset_hrs) 
    ret_utc = from_tz_utc.strftime("%Y-%m-%dT%H:%M:%S")
    return ret_utc

def get_local_string(from_utc):
    from_tz_utc = datetime.strptime(from_utc[:-4], "%Y-%m-%dT%H:%M:%S.%f") - timedelta(hours=offset_hrs)
    ret_local = from_tz_utc.strftime("%Y/%m/%d %H:%M")
    return ret_local

def run_query(group_text, from_ts, to_ts):
    time_from=get_utc_string(from_ts)
    time_to=get_utc_string(to_ts)

    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('saubury-001')

    fe = Key('time').between(time_from, time_to);
    response = table.scan(FilterExpression=fe)
    items = response['Items']

    
    for item in items:
        lat_array.append(item['Latitude'])
        lon_array.append(item['Longitude'])
        vistime_array.append(get_local_string(item['time']))
        group_array.append(group_text)


lat_array = []
lon_array = []
vistime_array = []
group_array = []

#run_query('out', '2021-01-01 14:00', '2021-01-01 15:20')
#run_query('back', '2021-01-01 15:20', '2021-01-01 16:20')
#run_query('front garden A', '2021-01-04 13:00', '2021-01-04 13:23')
#run_query('back garden', '2021-01-04 13:26', '2021-01-04 13:38')
#run_query('front garden A', '2021-01-04 13:39', '2021-01-04 14:48')

mydict = {'lat': lat_array, 'lon': lon_array, 'time' : vistime_array, 'group' : group_array}
mydf = pd.DataFrame(data=mydict)
#print(mydf)

pyo.init_notebook_mode()
fig = px.line_mapbox(mydf, lat="lat", lon="lon", color="group", hover_name="time", zoom=3, height=300)
fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=12, mapbox_center_lat = -33.8,
    margin={"r":0,"t":0,"l":0,"b":0})

fig.show()